Proses pemindahan data ke label yang benar, menghapus data-data yang tidak relevan, dan menambahkan data augmentasi yang diperoleh/diproses dari data train

# Import Library/Package

In [ ]:
import pandas as pd
import io
import os
import shutil
from pathlib import Path

# 1. Pemindahan Data

In [ ]:
data = """file_name,correct_class,class
balinese_train_000651.jpg,javanese,balinese
balinese_train_000760.jpg,batak,balinese
balinese_train_000761.jpg,batak,balinese
balinese_train_000762.jpg,batak,balinese
balinese_train_000763.jpg,dayak,balinese
balinese_train_000764.jpg,dayak,balinese
balinese_train_000765.jpg,javanese,balinese
balinese_train_000766.jpg,javanese,balinese
balinese_train_000767.jpg,javanese,balinese
balinese_train_000768.jpg,javanese,balinese
balinese_train_000769.jpg,javanese,balinese
balinese_train_000770.jpg,javanese,balinese
balinese_train_000771.jpg,minangkabau,balinese
balinese_train_000772.jpg,minangkabau,balinese
balinese_train_000773.jpg,minangkabau,balinese
balinese_train_000774.jpg,minangkabau,balinese
balinese_train_000776.jpg,minangkabau,balinese
batak_train_000001.jpg,balinese,batak
batak_train_000002.jpg,balinese,batak
batak_train_000006.jpg,balinese,batak
batak_train_000007.jpg,balinese,batak
batak_train_000010.jpg,javanese,batak
batak_train_000040.jpg,balinese,batak
batak_train_000062.jpg,minangkabau,batak
batak_train_000071.jpg,balinese,batak
batak_train_000086.jpg,balinese,batak
batak_train_000091.jpg,dayak,batak
batak_train_000092.jpg,javanese,batak
batak_train_000093.jpg,javanese,batak
batak_train_000094.jpg,minangkabau,batak
dayak_train_000004.jpg,minangkabau,dayak
dayak_train_000019.jpg,balinese,dayak
dayak_train_000052.jpg,javanese,dayak
javanese_train_000007.jpg,balinese,javanese
javanese_train_000009.jpg,balinese,javanese
javanese_train_000012.jpg,batak,javanese
javanese_train_000015.jpg,dayak,javanese
javanese_train_000025.jpg,balinese,javanese
javanese_train_000032.jpg,balinese,javanese
javanese_train_000041.jpg,balinese,javanese
javanese_train_000048.jpg,balinese,javanese
javanese_train_000050.jpg,balinese,javanese
javanese_train_000077.jpg,balinese,javanese
javanese_train_000085.jpg,dayak,javanese
javanese_train_000099.jpg,balinese,javanese
javanese_train_000111.jpg,batak,javanese
javanese_train_000122.jpg,balinese,javanese
javanese_train_000202.jpg,balinese,javanese
javanese_train_000213.jpg,batak,javanese
javanese_train_000245.jpg,minangkabau,javanese
javanese_train_000246.jpg,minangkabau,javanese
javanese_train_000248.jpg,minangkabau,javanese
javanese_train_000249.jpg,minangkabau,javanese
minangkabau_train_000005.jpg,balinese,minangkabau
minangkabau_train_000008.jpg,balinese,minangkabau
minangkabau_train_000010.jpg,balinese,minangkabau
minangkabau_train_000017.jpg,javanese,minangkabau
minangkabau_train_000021.jpg,javanese,minangkabau
minangkabau_train_000066.jpg,balinese,minangkabau
minangkabau_train_000079.jpg,balinese,minangkabau
minangkabau_train_000117.jpg,balinese,minangkabau
minangkabau_train_000180.jpg,javanese,minangkabau
minangkabau_train_000186.jpg,balinese,minangkabau
minangkabau_train_000200.jpg,javanese,minangkabau
minangkabau_train_000299.jpg,balinese,minangkabau
minangkabau_train_000411.jpg,batak,minangkabau
minangkabau_train_000416.jpg,javanese,minangkabau
minangkabau_train_000459.jpg,javanese,batak
batak_train_000059.jpg,minangkabau,batak
batak_train_000067.jpg,minangkabau,batak
batak_train_000095.jpg,minangkabau,batak
"""

# Menggunakan io.StringIO untuk membaca string sebagai file CSV
move_image = pd.read_csv(io.StringIO(data))

# Tampilkan 5 baris pertama
print(move_image.head())

# Tampilkan informasi DataFrame
print(move_image.info())

                   file_name correct_class     class
0  balinese_train_000651.jpg      javanese  balinese
1  balinese_train_000760.jpg         batak  balinese
2  balinese_train_000761.jpg         batak  balinese
3  balinese_train_000762.jpg         batak  balinese
4  balinese_train_000763.jpg         dayak  balinese
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71 entries, 0 to 70
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   file_name      71 non-null     object
 1   correct_class  71 non-null     object
 2   class          71 non-null     object
dtypes: object(3)
memory usage: 1.8+ KB
None


In [ ]:
# Gunakan dataframe move_image yang sudah didefinisikan sebelumnya
df = move_image

# Path ke folder Train
train_folder = "Train/Train"

# Debug: Cek struktur folder Train
print("Struktur folder Train:")
if os.path.exists(train_folder):
    for subfolder in os.listdir(train_folder):
        subfolder_path = os.path.join(train_folder, subfolder)
        if os.path.isdir(subfolder_path):
            file_count = len([f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"  {subfolder}: {file_count} file")
else:
    print(f"Folder {train_folder} tidak ditemukan!")

print(f"\nTotal file yang akan dipindahkan: {len(df)}")
print(f"Contoh data CSV:")
print(df.head())
print()

# Proses pemindahan gambar
moved_count = 0
not_found_count = 0
already_exists_count = 0

for idx, row in df.iterrows():
    file_name = row['file_name']
    original_class = row['class']  # kelas asal
    correct_class = row['correct_class']  # kelas tujuan
    
    # Cari file di kelas asal
    src_path = os.path.join(train_folder, original_class, file_name)
    
    # Jika tidak ditemukan di kelas asal, cari di semua subfolder
    if not os.path.exists(src_path):
        found_in_other = False
        for subfolder in os.listdir(train_folder):
            subfolder_path = os.path.join(train_folder, subfolder)
            if os.path.isdir(subfolder_path):
                alt_src_path = os.path.join(train_folder, subfolder, file_name)
                if os.path.exists(alt_src_path):
                    print(f"[{idx+1}/{len(df)}] File ditemukan di {subfolder} (bukan di {original_class}): {file_name}")
                    src_path = alt_src_path
                    found_in_other = True
                    break
        
        if not found_in_other:
            print(f"[{idx+1}/{len(df)}] File tidak ditemukan di manapun: {file_name}")
            not_found_count += 1
            continue
    
    # Proses pemindahan
    dst_path = os.path.join(train_folder, correct_class, file_name)
    
    # Pastikan folder tujuan ada
    os.makedirs(os.path.join(train_folder, correct_class), exist_ok=True)
    
    if os.path.exists(dst_path):
        print(f"[{idx+1}/{len(df)}] File sudah ada di tujuan: {file_name}")
        print(f"    Dari: {original_class} -> {correct_class}")
        already_exists_count += 1
    else:
        try:
            shutil.move(src_path, dst_path)
            print(f"[{idx+1}/{len(df)}] Berhasil memindahkan: {file_name}")
            print(f"    Dari: {original_class} -> {correct_class}")
            moved_count += 1
        except Exception as e:
            print(f"[{idx+1}/{len(df)}] Error memindahkan {file_name}: {e}")

print(f"\nProses pemindahan selesai!")
print(f"Berhasil dipindahkan: {moved_count}")
print(f"Sudah ada di tujuan: {already_exists_count}")
print(f"Tidak ditemukan: {not_found_count}")

# Tampilkan ringkasan perpindahan
print("\nRingkasan perpindahan:")
summary = df.groupby(['class', 'correct_class']).size().reset_index(name='count')
for _, row in summary.iterrows():
    print(f"Dari {row['class']} ke {row['correct_class']}: {row['count']} file")

Struktur folder Train:
  balinese: 776 file
  batak: 95 file
  dayak: 69 file
  javanese: 249 file
  minangkabau: 563 file

Total file yang akan dipindahkan: 71
Contoh data CSV:
                   file_name correct_class     class
0  balinese_train_000651.jpg      javanese  balinese
1  balinese_train_000760.jpg         batak  balinese
2  balinese_train_000761.jpg         batak  balinese
3  balinese_train_000762.jpg         batak  balinese
4  balinese_train_000763.jpg         dayak  balinese

[1/71] Berhasil memindahkan: balinese_train_000651.jpg
    Dari: balinese -> javanese
[2/71] Berhasil memindahkan: balinese_train_000760.jpg
    Dari: balinese -> batak
[3/71] Berhasil memindahkan: balinese_train_000761.jpg
    Dari: balinese -> batak
[4/71] Berhasil memindahkan: balinese_train_000762.jpg
    Dari: balinese -> batak
[5/71] Berhasil memindahkan: balinese_train_000763.jpg
    Dari: balinese -> dayak
[6/71] Berhasil memindahkan: balinese_train_000764.jpg
    Dari: balinese -> dayak
[7

# 2. Penghapusan Data

In [ ]:
data = """file_name,class
balinese_train_000008.jpg,balinese
balinese_train_000076.jpg,balinese
balinese_train_000094.jpg,balinese
balinese_train_000115.jpg,balinese
balinese_train_000192.jpg,balinese
balinese_train_000203.jpg,balinese
balinese_train_000207.jpg,balinese
balinese_train_000233.jpg,balinese
balinese_train_000256.jpg,balinese
balinese_train_000278.jpg,balinese
balinese_train_000301.jpg,balinese
balinese_train_000334.jpg,balinese
balinese_train_000345.jpg,balinese
balinese_train_000361.jpg,balinese
balinese_train_000363.jpg,balinese
balinese_train_000396.jpg,balinese
balinese_train_000416.jpg,balinese
balinese_train_000417.jpg,balinese
balinese_train_000423.jpg,balinese
balinese_train_000425.jpg,balinese
balinese_train_000426.jpg,balinese
balinese_train_000495.jpg,balinese
balinese_train_000500.jpg,balinese
balinese_train_000526.jpg,balinese
balinese_train_000528.jpg,balinese
balinese_train_000544.jpg,balinese
balinese_train_000545.jpg,balinese
balinese_train_000667.jpg,balinese
balinese_train_000708.jpg,balinese
balinese_train_000721.jpg,balinese
balinese_train_000749.jpg,balinese
balinese_train_000756.jpg,balinese
balinese_train_000775.jpg,balinese
batak_train_000004.jpg,batak
batak_train_000005.jpg,batak
batak_train_000032.jpg,batak
batak_train_000072.jpg,batak
dayak_train_000058.jpg,dayak
javanese_train_000142.jpg,javanese
javanese_train_000158.jpg,javanese
javanese_train_000158.jpg,javanese
javanese_train_000169.jpg,javanese
javanese_train_000244.jpg,javanese
javanese_train_000244.jpg,javanese
minangkabau_train_000003.jpg,minangkabau
minangkabau_train_000023.jpg,minangkabau
minangkabau_train_000024.jpg,minangkabau
minangkabau_train_000025.jpg,minangkabau
minangkabau_train_000026.jpg,minangkabau
minangkabau_train_000029.jpg,minangkabau
minangkabau_train_000042.jpg,minangkabau
minangkabau_train_000045.jpg,minangkabau
minangkabau_train_000046.jpg,minangkabau
minangkabau_train_000047.jpg,minangkabau
minangkabau_train_000074.jpg,minangkabau
minangkabau_train_000083.jpg,minangkabau
minangkabau_train_000097.jpg,minangkabau
minangkabau_train_000111.jpg,minangkabau
minangkabau_train_000114.jpg,minangkabau
minangkabau_train_000154.jpg,minangkabau
minangkabau_train_000163.jpg,minangkabau
minangkabau_train_000171.jpg,minangkabau
minangkabau_train_000177.jpg,minangkabau
minangkabau_train_000178.jpg,minangkabau
minangkabau_train_000192.jpg,minangkabau
minangkabau_train_000195.jpg,minangkabau
minangkabau_train_000209.jpg,minangkabau
minangkabau_train_000211.jpg,minangkabau
minangkabau_train_000216.jpg,minangkabau
minangkabau_train_000219.jpg,minangkabau
minangkabau_train_000223.jpg,minangkabau
minangkabau_train_000224.jpg,minangkabau
minangkabau_train_000229.jpg,minangkabau
minangkabau_train_000230.jpg,minangkabau
minangkabau_train_000237.jpg,minangkabau
minangkabau_train_000249.jpg,minangkabau
minangkabau_train_000250.jpg,minangkabau
minangkabau_train_000253.jpg,minangkabau
minangkabau_train_000258.jpg,minangkabau
minangkabau_train_000259.jpg,minangkabau
minangkabau_train_000268.jpg,minangkabau
minangkabau_train_000281.jpg,minangkabau
minangkabau_train_000286.jpg,minangkabau
minangkabau_train_000301.jpg,minangkabau
minangkabau_train_000306.jpg,minangkabau
minangkabau_train_000307.jpg,minangkabau
minangkabau_train_000316.jpg,minangkabau
minangkabau_train_000326.jpg,minangkabau
minangkabau_train_000345.jpg,minangkabau
minangkabau_train_000346.jpg,minangkabau
minangkabau_train_000364.jpg,minangkabau
minangkabau_train_000375.jpg,minangkabau
minangkabau_train_000386.jpg,minangkabau
minangkabau_train_000387.jpg,minangkabau
minangkabau_train_000388.jpg,minangkabau
minangkabau_train_000398.jpg,minangkabau
minangkabau_train_000399.jpg,minangkabau
minangkabau_train_000414.jpg,minangkabau
minangkabau_train_000419.jpg,minangkabau
minangkabau_train_000428.jpg,minangkabau
minangkabau_train_000429.jpg,minangkabau
minangkabau_train_000431.jpg,minangkabau
minangkabau_train_000441.jpg,minangkabau
minangkabau_train_000453.jpg,minangkabau
minangkabau_train_000455.jpg,minangkabau
minangkabau_train_000467.jpg,minangkabau
minangkabau_train_000497.jpg,minangkabau
minangkabau_train_000499.jpg,minangkabau
minangkabau_train_000502.jpg,minangkabau
minangkabau_train_000506.jpg,minangkabau
minangkabau_train_000507.jpg,minangkabau
minangkabau_train_000510.jpg,minangkabau
minangkabau_train_000520.jpg,minangkabau
minangkabau_train_000523.jpg,minangkabau
minangkabau_train_000526.jpg,minangkabau
minangkabau_train_000530.jpg,minangkabau
minangkabau_train_000536.jpg,minangkabau
minangkabau_train_000555.jpg,minangkabau
minangkabau_train_000559.jpg,minangkabau
minangkabau_train_000560.jpg,minangkabau
balinese_train_000559.jpg,balinese
balinese_train_000585.jpg,balinese
balinese_train_000589.jpg,balinese
balinese_train_000600.jpg,balinese
balinese_train_000606.jpg,balinese
balinese_train_000629.jpg,balinese
minangkabau_train_000378.jpg,minangkabau
minangkabau_train_000336.jpg,minangkabau
"""

# Menggunakan io.StringIO untuk membaca string sebagai file CSV
deleted_images = pd.read_csv(io.StringIO(data))

# Tampilkan 5 baris pertama
print(deleted_images.head())

# Tampilkan informasi DataFrame
print(deleted_images.info())

                   file_name     class
0  balinese_train_000008.jpg  balinese
1  balinese_train_000076.jpg  balinese
2  balinese_train_000094.jpg  balinese
3  balinese_train_000115.jpg  balinese
4  balinese_train_000192.jpg  balinese
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128 entries, 0 to 127
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  128 non-null    object
 1   class      128 non-null    object
dtypes: object(2)
memory usage: 2.1+ KB
None


In [ ]:
# Gunakan dataframe deleted_images yang sudah didefinisikan sebelumnya
df_deleted = deleted_images

# Debug: cek struktur folder
print("Checking folder structure:")
possible_paths = ['Train', 'Train/Train', 'All/Train']
train_folder = None

for path in possible_paths:
    if os.path.exists(path):
        print(f"{path} folder exists")
        # Cek apakah ada subfolder kelas di dalamnya
        subdirs = [d for d in os.listdir(path) if os.path.isdir(os.path.join(path, d))]
        print(f"Subfolders in {path}: {subdirs}")
        
        # Jika ada subfolder yang terlihat seperti kelas, gunakan path ini
        class_names = ['balinese', 'batak', 'dayak', 'javanese', 'minangkabau']
        if any(class_name in subdirs for class_name in class_names):
            train_folder = path
            print(f"Found class folders in {path}, using this as train_folder")
            break
        elif path == 'Train/Train':
            train_folder = path
            print(f"Using nested Train/Train structure")
            break

if train_folder is None:
    print("No suitable train folder found!")
    train_folder = 'Train/Train'  # Default fallback

print(f"\nUsing train folder: {train_folder}")

# Tampilkan struktur folder yang akan digunakan
if os.path.exists(train_folder):
    print(f"\nContents of {train_folder}:")
    for item in os.listdir(train_folder):
        item_path = os.path.join(train_folder, item)
        if os.path.isdir(item_path):
            file_count = len([f for f in os.listdir(item_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
            print(f"  {item}/: {file_count} images")

# Debug: cek nama file dari dataframe vs file asli
print("\nFirst 5 filenames from deleted_images dataframe:")
for i, filename in enumerate(df_deleted['file_name'][:5]):
    print(f"'{filename}' (length: {len(filename)})")

# Counter untuk tracking
deleted_count = 0
not_found_count = 0
total_files = len(df_deleted['file_name'])

print(f"\nStarting deletion process...")
print(f"Total files to delete: {total_files}")

# Iterate over each filename in df_deleted and remove the file if it exists
for idx, filename in enumerate(df_deleted['file_name'], 1):
    # Search for the file in all subfolders (class folders)
    file_found = False
    for root, dirs, files in os.walk(train_folder):
        if filename in files:
            file_path = os.path.join(root, filename)
            if os.path.exists(file_path):
                os.remove(file_path)
                print(f"[{idx}/{total_files}] Deleted: {file_path}")
                file_found = True
                deleted_count += 1
                break
    
    if not file_found:
        print(f"[{idx}/{total_files}] File not found: '{filename}'")
        not_found_count += 1

# Summary
print(f"\n=== SUMMARY ===")
print(f"Total files to delete: {total_files}")
print(f"Successfully deleted: {deleted_count}")
print(f"Files not found: {not_found_count}")
print(f"Success rate: {deleted_count/total_files*100:.1f}%")

Checking folder structure:
Train folder exists
Subfolders in Train: ['Train']
Train/Train folder exists
Subfolders in Train/Train: ['balinese', 'batak', 'dayak', 'javanese', 'minangkabau']
Found class folders in Train/Train, using this as train_folder

Using train folder: Train/Train

Contents of Train/Train:
  balinese/: 786 images
  batak/: 86 images
  dayak/: 71 images
  javanese/: 246 images
  minangkabau/: 563 images

First 5 filenames from deleted_images dataframe:
'balinese_train_000008.jpg' (length: 25)
'balinese_train_000076.jpg' (length: 25)
'balinese_train_000094.jpg' (length: 25)
'balinese_train_000115.jpg' (length: 25)
'balinese_train_000192.jpg' (length: 25)

Starting deletion process...
Total files to delete: 128
[1/128] Deleted: Train/Train\balinese\balinese_train_000008.jpg
[2/128] Deleted: Train/Train\balinese\balinese_train_000076.jpg
[3/128] Deleted: Train/Train\balinese\balinese_train_000094.jpg
[4/128] Deleted: Train/Train\balinese\balinese_train_000115.jpg
[5/128

# 3. Data Augmentasi

In [ ]:
data = """file_name,class
balinese_train_augmented_001.png,balinese
balinese_train_augmented_002.png,balinese
balinese_train_augmented_003.jpg,balinese
balinese_train_augmented_004.png,balinese
balinese_train_augmented_005.png,balinese
balinese_train_augmented_006.png,balinese
balinese_train_augmented_007.png,balinese
balinese_train_augmented_008.png,balinese
balinese_train_augmented_009.png,balinese
balinese_train_augmented_010.png,balinese
balinese_train_augmented_011.png,balinese
balinese_train_augmented_012.jpg,balinese
balinese_train_augmented_013.png,balinese
balinese_train_augmented_014.png,balinese
balinese_train_augmented_015.png,balinese
balinese_train_augmented_016.jpg,balinese
balinese_train_augmented_017.jpg,balinese
dayak_train_augmented_001.png,dayak
javanese_train_augmented_001.png,javanese
javanese_train_augmented_002.png,javanese
javanese_train_augmented_003.png,javanese
javanese_train_augmented_004.png,javanese
javanese_train_augmented_005.png,javanese
javanese_train_augmented_006.jpg,javanese
javanese_train_augmented_007.png,javanese
javanese_train_augmented_008.png,javanese
javanese_train_augmented_009.png,javanese
javanese_train_augmented_010.png,javanese
minangkabau_train_augmented_001.png,minangkabau
minangkabau_train_augmented_002.png,minangkabau
minangkabau_train_augmented_003.png,minangkabau
minangkabau_train_augmented_004.png,minangkabau
"""

# Menggunakan io.StringIO untuk membaca string sebagai file CSV
augmented_image = pd.read_csv(io.StringIO(data))

# Tampilkan 5 baris pertama
print(augmented_image.head())

# Tampilkan informasi DataFrame
print(augmented_image.info())

                          file_name     class
0  balinese_train_augmented_001.png  balinese
1  balinese_train_augmented_002.png  balinese
2  balinese_train_augmented_003.jpg  balinese
3  balinese_train_augmented_004.png  balinese
4  balinese_train_augmented_005.png  balinese
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32 entries, 0 to 31
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   file_name  32 non-null     object
 1   class      32 non-null     object
dtypes: object(2)
memory usage: 640.0+ bytes
None


In [ ]:
df_augmented = augmented_image

# Define source and destination directories
source_dir = 'D:/Lomba/LOGIKA-UI/augmented-image'  # Change this to your source folder path
destination_base_dir = 'D:/Lomba/LOGIKA-UI/Train/Train'  # Change this to your destination base folder

# Create destination folders if they don't exist
classes = df_augmented['class'].unique()
for cls in classes:
    os.makedirs(os.path.join(destination_base_dir, cls), exist_ok=True)

# Move files to their corresponding class folders
for _, row in df_augmented.iterrows():
    file_name = row['file_name']
    class_name = row['class']
    
    source_path = os.path.join(source_dir, file_name)
    destination_path = os.path.join(destination_base_dir, class_name, file_name)
    
    if os.path.exists(source_path):
        shutil.copy2(source_path, destination_path)  # Use copy2 to preserve metadata
        print(f"Moved {file_name} to {class_name} folder")
    else:
        print(f"Warning: {file_name} not found in source directory")

print(f"Done moving {df_augmented.shape[0]} files to their class folders")

Moved balinese_train_augmented_001.png to balinese folder
Moved balinese_train_augmented_002.png to balinese folder
Moved balinese_train_augmented_003.jpg to balinese folder
Moved balinese_train_augmented_004.png to balinese folder
Moved balinese_train_augmented_005.png to balinese folder
Moved balinese_train_augmented_006.png to balinese folder
Moved balinese_train_augmented_007.png to balinese folder
Moved balinese_train_augmented_008.png to balinese folder
Moved balinese_train_augmented_009.png to balinese folder
Moved balinese_train_augmented_010.png to balinese folder
Moved balinese_train_augmented_011.png to balinese folder
Moved balinese_train_augmented_012.jpg to balinese folder
Moved balinese_train_augmented_013.png to balinese folder
Moved balinese_train_augmented_014.png to balinese folder
Moved balinese_train_augmented_015.png to balinese folder
Moved balinese_train_augmented_016.jpg to balinese folder
Moved balinese_train_augmented_017.jpg to balinese folder
Moved dayak_tr